In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

In [ ]:
#Import libraries & packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
import random
from google.colab.patches import cv2_imshow
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from PIL import Image, ImageOps
from sklearn.utils import shuffle
import pathlib
import os
import cv2
import math
from math import log10, sqrt
import glob
from matplotlib import pyplot as plt
%matplotlib inline

import time

from sklearn.model_selection import train_test_split
import skimage
from skimage import data
from skimage import util
from skimage.filters.rank import gradient
from skimage.color import rgb2gray
from skimage.morphology import disk
from scipy import ndimage, misc
import scipy.misc
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras import layers 
import tensorflow as tf
from tensorflow import distribute
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, Reshape, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adadelta, RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, UpSampling2D
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, Conv2D, Reshape, Conv2DTranspose
#from tensorflow.keras.layers import Dense, Input, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import structural_similarity, peak_signal_noise_ratio, mean_squared_error, mean_squared_error, normalized_root_mse

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
images = sorted(os.listdir('gdrive/My Drive/images/chestxray'))
image_data = []

for im in images:
  img = image.load_img('gdrive/My Drive/images/chestxray/'+im, target_size = (128,128), color_mode = 'grayscale')
  img = image.img_to_array(img)
  img = img/255
  image_data.append(img)

image_dataset = np.array(image_data)

In [ ]:
#Subplotting images
def plot_img(dataset):
  f, ax = plt.subplots(1,5)
  f.set_size_inches(40,20)
  for i in range(5,10):
    ax[i-5].imshow(dataset[i].reshape(128, 128), cmap='gray')
  plt.show()

In [ ]:
plot_img(image_dataset)

In [ ]:
image_data = Image.open('/content/gdrive/My Drive/images/chestxray/000001-4.png').convert('L')

#Subplotting images
width, height = image_data.size
plt.subplots(figsize=(10, 6))

print('---------------------------------')
print('| original image size: %d x %d'%(width,height), '|')
print('---------------------------------')
plt.imshow(image_data)
plt.show()

#Resize image to 256x256
newwidth = 256
newheight = 256
#convert to numpy array
scaled = np.array(image_data.resize((newwidth, newheight)))
plt.subplots(figsize=(10, 6))
print('-----------------------------')
print('| new image size: %d x %d'%(newwidth,newheight), '|')
print('-----------------------------')
plt.imshow(scaled)
plt.show()

In [ ]:
def noise(image,level):
    """Adds gaussian random noise
    arguments: image, level (level of noise required)
    returns: image with level of gaussian noise added
    """
    temp=image+level*np.random.normal(0, 128, size=(image.shape[0],image.shape[1]))
    #removing negative pixels (set to zero) and normalising back to 255
    regular= temp*(temp>0)
    mynorm=255./np.max(temp)
    #convert back to integer type
    temp = np.around(mynorm*regular).astype('int')
    return(temp)

#add chosen % of noise to the image
amount=0.1 #% of noise required - 0.1 = 10%, 0.05 = 5% etc.
noisy_img=noise(scaled,amount)
#show the transformation
plt.subplots(figsize=(10, 6))
print('--------------------------')
print('Image with %.4f noise'%(amount),':')
print('--------------------------')
plt.imshow(noisy_img)
plt.show()

In [ ]:
#using Keras' image_data_generator to create a dataset

#add a channel index at the end of each np.array
channel1 = np.expand_dims(scaled, 2)
#define samples variable as being just one image and add a dimension for this
samples= np.expand_dims(channel1, 0)

#image data augmentation generator
datagen = ImageDataGenerator(rotation_range=10,
                             height_shift_range=0.2,
                             width_shift_range=0.2,
                             fill_mode='constant',)
datagen.fit(samples)

#create the new dataset including clean, noisy, and very noisy images
maxsize=1000 #required size of the dataset

#storage arrays
clean=[]
noisy=[]

#instanciating the generator
it = datagen.flow(samples, batch_size=1)

#add the instances to storage arrays
for i in range(maxsize):
  new_image=it.next()[0]

# normalising the image
  clean.append(new_image.astype('float32')/255.)
# noise addition to subset of images and normalising 
  noisy.append(noise(new_image[:,:,0],0.1).astype('float32')/255.)

#re-shaping the sets so that they are in the sample, x,y, channel order
created_set=np.reshape(clean, [-1, 256,256, 1])
noisycreated_set=np.reshape(noisy, [-1, 256,256, 1])

In [ ]:
#show outputs

#image 16, channel 0: clean (augmented) & noisy (augmented)
plt.imshow(created_set[16,:,:,0])
plt.show()
plt.imshow(noisycreated_set[16,:,:,0])
plt.show()

#image 97, channel 0: clean (augmented) & noisy (augmented)
plt.imshow(created_set[97,:,:,0])
plt.show()
plt.imshow(noisycreated_set[97,:,:,0])
plt.show()

In [ ]:
#splitting the dataset into training and test (70/30)
X_train, X_test = train_test_split(created_set, test_size = 0.3, random_state = 42)
X_train_noisy, X_test_noisy = train_test_split(noisycreated_set, test_size = 0.3, random_state = 42)

In [ ]:
#define custom SSIM Loss function
def SSIMLoss(y_true, y_pred):
    """Defines a custom loss function for SSIM measurement models
    Arguments: y_true, the true data, and y_pred, calculated prediction
    Returns: SSIMLoss loss function
    """
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

In [ ]:
#network adapted from https://keras.io/examples/vision/autoencoder/

input = Input(shape=channel1.shape)
# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
#x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
#x = MaxPooling2D((2, 2), padding='same)(x)
#x = Conv2D(8, (3, 3), activation='relu, padding='same)(x)
#x = MaxPooling2D((2, 2), padding='same)(x)

# Decoder
#x = Conv2DTranspose(8, (3, 3), strides=2, activation='relu', padding='same')(x) 
#reduce strides (2 -> 1) to match removal of max pooling layers on encoder side
x = Conv2DTranspose(16, (3, 3), strides=1, activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), strides=1, activation='relu', padding='same')(x)
x = Conv2D(1, (3, 3), activation='relu', padding='same')(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer='adam', loss=SSIMLoss)
autoencoder.summary()

In [ ]:
batch_size = 16

#training the model
history = autoencoder.fit(X_train_noisy, X_train, validation_data=(X_test_noisy, X_test), epochs=5, shuffle=True, batch_size=batch_size)

In [ ]:
# Plot model losses

loss_plt = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
#plt.ylim([0, 1])
loss_plt



In [ ]:
fig, axs = plt.subplots(figsize=(5,5))

axs.plot(history.history['loss'])
axs.plot(history.history['val_loss'])
axs.title.set_text('Training Loss vs Validation Loss')
axs.set_xlabel('Epochs')
axs.set_ylabel('Loss')
axs.legend(['Train','Val'])

In [ ]:
# Predict AE output from noisy image
noisy_decoded = autoencoder.predict(samples)
plt.subplots(figsize=(10, 6))
print('---------------------')
print('Denoised image output:')
print('---------------------')
plt.imshow(noisy_decoded[0].reshape(256,256))
plt.show()

In [ ]:
def sharpness_measure(imageA, imageB, title): 
    #adapted from https://dsp.stackexchange.com/questions/35490/how-to-locally-quantify-the-sharpness-of-an-image
    """function to show the sharpness of 2 side-by-side images as gradient maps
    Arguments: 2 images, title
    Returns: Plots of the 2 images with the difference in contrast (i.e., sharpness) measured by gradient
    """
    imageA_gray = rgb2gray(imageA)
    imageB_gray = rgb2gray(imageB)
    disk_matrix = disk(5) #matrix with a disk shape
    imageA_sharpness = gradient(imageA_gray, disk_matrix)
    imageB_sharpness = gradient(imageB_gray, disk_matrix)
    #Show first image
    plt.imshow(imageA_sharpness, cmap='viridis')
    plt.axis('off')
    plt.colorbar()
    plt.show()
    #show second image
    plt.imshow(imageB_sharpness, cmap='viridis')
    plt.axis('off')
    plt.colorbar()
    plt.show()

In [ ]:
def compare(imageA, imageB, title):

  psnr = peak_signal_noise_ratio(imageA, imageB) #, data_range = 255

  ssim = structural_similarity(imageA, imageB) #, multichannel=True, data_range = 255

  mse = mean_squared_error(imageA, imageB)

  nrmse = normalized_root_mse(imageA, imageB) #,  normalization='euclidean'

  # setup the figure
  fig = plt.figure(title)
  print('PSNR: %.2f, SSIM: %.2f, MSE: %.6f, NRMSE: %.4f' % (psnr, ssim, mse, nrmse))
  plt.suptitle('PSNR: %.2f, SSIM: %.2f, MSE: %.6f, NRMSE: %.4f' % (psnr, ssim, mse, nrmse))

  # show first image
  ax = fig.add_subplot(1, 2, 1)
  plt.imshow(imageA)
  plt.axis('off')

  # show the second image
  ax = fig.add_subplot(1, 2, 2)
  plt.imshow(imageB)
  plt.axis('off')

  # show the images
  plt.show()

  #return psnr, ssim, mse, nrmse

In [ ]:
#normalising images for comparison
#Normalised_reference = scaled.astype('float32')/255.
#Normalised_decoded = noisy_decoded[0].reshape(256,256)/np.max(noisy_decoded[0])
#Normalised_noisy_original = noisy_img.astype('float32')/255.

In [ ]:
#normalising images for comparison
Normalised_reference = scaled.astype('float32')/255.
Normalised_decoded = noisy_decoded[0].reshape(256,256)/np.max(noisy_decoded[0])
Normalised_noisy_original = noisy_img.astype('float32')/255.

In [ ]:
#compare images clean reference with de-noised
plt.subplots(figsize=(10, 6))
compare (Normalised_reference, Normalised_decoded,'')
#ideally want mse to be close to 0, ssim to be close to 1 

In [ ]:
#Compare images clean reference with noisy original
plt.subplots(figsize=(10, 6))
compare(Normalised_reference, Normalised_noisy_original,'')

In [ ]:
#Sharpness measure: shows the sharpness of the original non-noisy image vs the output as a gradient
sharpness_measure(Normalised_reference, Normalised_decoded,'') 

In [ ]:
median_filter = ndimage.median_filter(X_train[0].reshape(256, 256), size= 5) 

bilateral_filter = cv2.bilateralFilter(X_train[0].reshape(256, 256), 5, 50, 50)

In [ ]:
# show first image
ax = fig.add_subplot(1, 2, 1)
plt.imshow(median_filter)
plt.axis('off')

# show the second image
ax = fig.add_subplot(1, 2, 2)
plt.imshow(bilateral_filter)
plt.axis('off')

# show the images
plt.show()